---
jupyter: python3
toc: true
toc-depth: 3
toc-expand: true
number-sections: true
title: Pandas_04_Dataframe 결합
date: 2021-11-05 00:04
categories: pandas
author: limyj0708
comments:
  giscus:
    repo: limyj0708/blog
format:
    html:
        page-layout: full
---

In [2]:
import pandas as pd
import numpy as np
import copy
from IPython.display import display_html, display

# 위아래로 붙이는 단순 결합의 경우, 어떤 방식이 가장 빠른가?
- 결론부터 말하자면 데이터를 Dictionary의 리스트로 관리하다가 마지막에 Dataframe으로 만드는 것이 가장 빠르다.
  - [https://stackoverflow.com/questions/57000903/what-is-the-fastest-and-most-efficient-way-to-append-rows-to-a-dataframe](https://stackoverflow.com/questions/57000903/what-is-the-fastest-and-most-efficient-way-to-append-rows-to-a-dataframe)
  ```Python
    start_time = time.time()
    dictinary_list = []
    for i in range(0, end_value, 1):
        dictionary_data = {k: random.random() for k in range(30)}
        dictionary_list.append(dictionary_data)

    df_final = pd.DataFrame.from_dict(dictionary_list)

    end_time = time.time()
    print('Execution time = %.6f seconds' % (end_time-start_time))
  ```
  
  - 그럼 리스트 합치는 건 뭐가 제일 빠르지?
    - [https://www.realpythonproject.com/day15-the-fastest-way-to-combine-lists-in-python/](https://www.realpythonproject.com/day15-the-fastest-way-to-combine-lists-in-python/)
    - append() is the fastest but it doesn’t combine the elements of both the lists. **The + operator seems to be the ideal option.** However, this has been done on a comparatively smaller dataset and results may vary when you try it on your own.
- 인생은 항상 원하는대로 흘러가지 않기에, 다른 방법도 알아보자.

## concat
`pandas.concat(objs, axis=0, join='outer', ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, sort=False, copy=True)`

In [5]:
s1 = pd.Series(['a', 'b'])
s2 = pd.Series(['c', 'd'])
pd.concat([s1, s2]) # Series 두 개 합치기

0    a
1    b
0    c
1    d
dtype: object

In [6]:
pd.concat([s1, s2], ignore_index=True) # 합치면서 index 새로 만들어줌

0    a
1    b
2    c
3    d
dtype: object

In [7]:
s3 = pd.concat([s1, s2], keys=['s1', 's2']) # 최외각 레벨에 새로운 index를 만들어줌
print(s3)
print(s3['s1']) 
print(s3['s2'][0]) # 이렇게 조회가능

s1  0    a
    1    b
s2  0    c
    1    d
dtype: object
0    a
1    b
dtype: object
c


In [8]:
s3 = pd.concat([s1, s2], keys=['s1', 's2'], names=['Series name', 'Row ID'])
# index에 이름 붙이기
print(s3)
print(s3.index)
print(s3.index.names)

Series name  Row ID
s1           0         a
             1         b
s2           0         c
             1         d
dtype: object
MultiIndex([('s1', 0),
            ('s1', 1),
            ('s2', 0),
            ('s2', 1)],
           names=['Series name', 'Row ID'])
['Series name', 'Row ID']


In [9]:
df1 = pd.DataFrame([['a', 1], ['b', 2]], columns=['letter', 'number'])
print(df1)
df2 = pd.DataFrame([['c', 3], ['d', 4]], columns=['letter', 'number'])
print(df2)
pd.concat([df1, df2]) # Dataframe 합치기

  letter  number
0      a       1
1      b       2
  letter  number
0      c       3
1      d       4


,letter,number
0,a,1
1,b,2
0,c,3
1,d,4


In [10]:
df3 = pd.DataFrame([['c', 3, 'cat'], ['d', 4, 'dog']],
                   columns=['letter', 'number', 'animal'])
print(df3)
pd.concat([df1, df3], sort=False) # 한 쪽에 없는 컬럼의 값은 NaN으로 삽입됨

  letter  number animal
0      c       3    cat
1      d       4    dog


,letter,number,animal
0,a,1,NaN
1,b,2,NaN
0,c,3,cat
1,d,4,dog


In [11]:
pd.concat([df1, df3], join="inner") # join="inner"로 하면 양쪽에 다 있는 컬럼만 합쳐서 반환함

,letter,number
0,a,1
1,b,2
0,c,3
1,d,4


In [12]:
df4 = pd.DataFrame([['bird', 'polly'], ['monkey', 'george']],
                   columns=['animal', 'name'])
pd.concat([df1, df4], axis=1) # axis=1이면 컬럼을 붙임

,letter,number,animal,name
0,a,1,bird,polly
1,b,2,monkey,george


In [13]:
df4 = pd.DataFrame([['bird', 'polly'], ['monkey', 'george'], ['dog', 'sam']],
                   columns=['animal', 'name'])
pd.concat([df1, df4], axis=1) 
# axis=1이면 컬럼을 붙임
# 행의 수가 다르면, 행이 적은 쪽에 NaN이 삽입된 행이 추가됨

,letter,number,animal,name
0,a,1.0,bird,polly
1,b,2.0,monkey,george
2,NaN,NaN,dog,sam


In [14]:
df5 = pd.DataFrame([1], index=['a'])
df6 = pd.DataFrame([2], index=['a'])
pd.concat([df5, df6], verify_integrity=True)
# verify_integrity=True를 하면, index가 같은 것을 허용하지 않음.

ValueError: Indexes have overlapping values: Index(['a'], dtype='object')

# Merge : Database의 Join처럼 Dataframe 합치기
`DataFrame.merge(right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)`

- right : 합칠 Dataframe
- how : {‘left’, ‘right’, ‘outer’, ‘inner’, ‘cross’}, default ‘inner’
  - left: use only keys from left frame, similar to a SQL left outer join; preserve key order.
  - right: use only keys from right frame, similar to a SQL right outer join; preserve key order.
  - outer: use union of keys from both frames, similar to a SQL full outer join; sort keys lexicographically.
  - inner: use intersection of keys from both frames, similar to a SQL inner join; preserve the order of the left keys.
  - cross: creates the cartesian product from both frames, preserves the order of the left keys.
- on : label or list
  - 조인할 컬럼이나 인덱스 레벨의 이름. 두 Dataframe에 무조건 있어야 한다.
- left_on : label or list, or array-like
  - 왼쪽 Dataframe의 조인할 컬럼이나 인덱스 레벨의 이름.
- right_on : label or list, or array-like
  - 오른쪽 Dataframe의 조인할 컬럼이나 인덱스 레벨의 이름.
- left_index : bool, default False
  - 왼쪽 index를 조인의 key로 사용할까요?
  - MultiIndex인 경우, 상대 Dataframe의 key 수가 level의 수와 동일해야 함.
- right_index : bool, default False
  - 오른쪽 index를 조인의 key로 사용할까요?
  - MultiIndex인 경우, 상대 Dataframe의 key 수가 level의 수와 동일해야 함.
- sort : bool, default False
  - 조인 결과 Dataframe에서 key를 사전 순서(lexicographically)로 배열함
  - False인 경우, 조인 방법에 정의된 방법을 따라감
- suffixes : list-like, default is (“_x”, “_y”)
  - 컬럼에 접미사를 붙인다. 왼쪽 오른쪽 구분용.
  - 기본적으로 왼쪽에 _x, 오른쪽에 _y가 붙는다.
- copy : bool, default True
  - False면, 가능하면 복사를 피한다.
- indicator : bool or str, default False
- validate : str, optional
  - If specified, checks if merge is of specified type.
      - “one_to_one” or “1:1”: check if merge keys are unique in both left and right datasets.
      - “one_to_many” or “1:m”: check if merge keys are unique in left dataset.
      - “many_to_one” or “m:1”: check if merge keys are unique in right dataset.
      - “many_to_many” or “m:m”: allowed, but does not result in checks.

In [16]:
df1 = pd.DataFrame({'lkey': ['foo', 'bar', 'baz', 'foo'], 'value': [1, 2, 3, 5]})
df2 = pd.DataFrame({'rkey': ['foo', 'bar', 'baz', 'foo'], 'value': [5, 6, 7, 8]})
display(df1, df2)

,lkey,value
0,foo,1
1,bar,2
2,baz,3
3,foo,5


,rkey,value
0,foo,5
1,bar,6
2,baz,7
3,foo,8


In [17]:
df1.merge(df2, how='inner', on='a')
# a 컬럼을 키로 잡음. 두 Dataframe에 다 a 컬럼이 있어서 가능한것
# inner join

,a,b,c
0,foo,1,3


In [18]:
df1.merge(df2, how='left', on='a')
# left outer join

,a,b,c
0,foo,1,3.0
1,bar,2,NaN


In [19]:
df1.merge(df2, how='left', left_on='a', right_on='a')
# left outer join

,a,b,c
0,foo,1,3.0
1,bar,2,NaN


## Dataframe Join의 속도를 향상시키기 위해서는?

- [https://stackoverflow.com/questions/40860457/improve-pandas-merge-performance](https://stackoverflow.com/questions/40860457/improve-pandas-merge-performance)
- key를 index로 사용한다.
  - index 검색 시에는 hash table을 이용하기 때문
  - A short explanation why it is faster to merge by index instead of by a "normal" column: Indices have a hash table. Meaning you can look them up in amortized O(1). For a normal column you need O(n) in worst case, meaning merging two dfs with len n takes O(n^2) in worst case.
- join을 쓴다.
- concat을 쓴다.

**여기서의 결론** : key를 index로 사용한 후 join을 쓴다.

In [20]:
import random
df1 = pd.DataFrame({'uid_sample': random.sample(range(100000), 80000), 'value': random.sample(range(10000000), 80000)})
df2 = pd.DataFrame({'userId_sample2': random.sample(range(100000), 80000), 'value': random.sample(range(10000000), 80000)})
# 80000명의 정보를 담고 있는 두 Dataframe이 있다고 하자.
# uid_sample, userId_sample2를 key로 조인하고 싶다.

In [21]:
%%timeit
df1.merge(df2, how='left', left_on='uid_sample', right_on='userId_sample2')

7.02 ms ± 478 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
%%timeit
# key로 사용하려는 컬럼을 index로 할당
# 10%정도 빨라졌다.
df3 = df1.set_index('uid_sample')
df4 = df2.set_index('userId_sample2')
df3.merge(df4, right_index=True, left_index=True)

6.3 ms ± 195 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [23]:
%%timeit
# key로 사용하려는 컬럼을 index로 할당
# join 함수 사용
# 여기서 이미 3배 빨라졌다
df3 = df1.set_index('uid_sample')
df4 = df2.set_index('userId_sample2')
df3.join(df4, how='left', lsuffix='left', rsuffix='right')

2.08 ms ± 88.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
%%timeit
# inner, outer밖에 안 되는데 join보다 느리다.
df3 = df1.set_index('uid_sample')
df4 = df2.set_index('userId_sample2')
pd.concat([df3, df4], axis=1, join='inner')

6.12 ms ± 87 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Join : Merge보다 빠르다
`DataFrame.join(other, on=None, how='left', lsuffix='', rsuffix='', sort=False)`

- other : 다른 데이터프레임, 혹은 시리즈, 혹은 데이터프레임 리스트
  - 함수를 호출한 데이터프레임에 붙일 대상
- on : 조인 키가 될 컬럼 이름, 혹은 컬럼 이름 리스트(array-like 자료형이면 됨)
- how
  - left : 함수를 호출한 데이터프레임(caller)의 index를 조인 키로 사용. on에서 컬럼을 지정했을 경우, 그 컬럼을 사용
  - right : other 패러미터에 할당된 객체의 index를 사용
  - outer : outer join 실행 후, 사전 순으로 정렬함. 기본적으론 양쪽 다 index를 사용. on에서 지정하면 caller만 해당 컬럼 사용.
  - inner : inner join 실행. caller의 순서 보존됨.
  - cross : 양쪽의 곱집합 생성. left key(caller)의 순서 보존됨.
- lsuffix, rsuffix : join된 결과물 컬럼의 접미사 세팅.
- sort : TRUE면, join key의 사전 순서대로 정렬됨. FALSE면, how에서의 기본 처리방식을 따름.

In [18]:
caller = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3', 'K4', 'K5'], 'A': ['A0', 'A1', 'A2', 'A3', 'A4', 'A5']})
other = pd.DataFrame({'key': ['K0', 'K1', 'K2'], 'B': ['B0', 'B1', 'B2']})
other2 = pd.DataFrame({'key': ['K0', 'K1', 'K2'], 'C': ['C0', 'C1', 'C2']})

In [19]:
display(caller, other, other2)

,key,A
0,K0,A0
1,K1,A1
2,K2,A2
3,K3,A3
4,K4,A4
5,K5,A5


,key,B
0,K0,B0
1,K1,B1
2,K2,B2


,key,C
0,K0,C0
1,K1,C1
2,K2,C2


In [27]:
# 이러면 index 0,1,2,3,4,5 기준으로 join됨
caller.join(other, lsuffix='_caller', rsuffix='_other')

,key_caller,A,key_other,B
0,K0,A0,K0,B0
1,K1,A1,K1,B1
2,K2,A2,K2,B2
3,K3,A3,NaN,NaN
4,K4,A4,NaN,NaN
5,K5,A5,NaN,NaN


In [28]:
# set_index로 조인 키로 쓰고 싶은 컬럼을 index로 만들어주는 방법이 있음
caller.set_index('key').join(other.set_index('key'))

,A,B
key,,
K0,A0,B0
K1,A1,B1
K2,A2,B2
K3,A3,NaN
K4,A4,NaN
K5,A5,NaN


In [29]:
# 혹은, other만 조인 키로 쓰고 싶은 컬럼을 index로 만들어 주고,
# on에다 조인 키로 쓰고 싶은 caller의 컬럼을 할당하는 방법이 있음
caller.join(other.set_index('key'), on='key')

,key,A,B
0,K0,A0,B0
1,K1,A1,B1
2,K2,A2,B2
3,K3,A3,NaN
4,K4,A4,NaN
5,K5,A5,NaN


In [30]:
# 한 번에 여러 개의 Dataframe을 Join 할 때에는, index로 join하는 것만 지원한다.
# 즉, on을 쓰지 못한다는 이야기이다.
caller.set_index('key').join([other.set_index('key'), other2.set_index('key')])

,A,B,C
key,,,
K0,A0,B0,C0
K1,A1,B1,C1
K2,A2,B2,C2
K3,A3,NaN,NaN
K4,A4,NaN,NaN
K5,A5,NaN,NaN
